# Statisctical information, mean and std : 

This notebook computes the mean, std for the arrival time for each train_type, hour and station. Note, this notebook takes approximately 20 minutes to run.  

### Set up spark:

In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8875,application_1589299642358_3407,pyspark,idle,Link,Link,
8901,application_1589299642358_3433,pyspark,idle,Link,Link,
8919,application_1589299642358_3451,pyspark,idle,Link,Link,
8931,application_1589299642358_3463,pyspark,idle,Link,Link,
8933,application_1589299642358_3465,pyspark,idle,Link,Link,
8935,application_1589299642358_3467,pyspark,busy,Link,Link,
8945,application_1589299642358_3477,pyspark,idle,Link,Link,
8964,application_1589299642358_3497,pyspark,idle,Link,Link,
8969,application_1589299642358_3504,pyspark,idle,Link,Link,
8970,application_1589299642358_3505,pyspark,idle,Link,Link,


#### Imports:

In [2]:
import networkx as nx
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from networkx.algorithms.shortest_paths.weighted import dijkstra_path
from scipy.stats import norm

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9001,application_1589299642358_3539,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

In [4]:
%%send_to_spark -i username -t str -n username

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [5]:
stops_zurich = spark.read.format('orc').load("/user/{}/nodes.orc".format(username))\
                                        .select('stop_id').distinct().rdd.flatMap(lambda x:x).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Taken from find_train_type_correspondace.ipynb
replace_actual = {
    'BUS': 'Bus', # Buses
    'B': 'Bus',
    'NFB': 'Bus',
    'KB': 'Bus',
    'BAT': 'Bus',
    'Trm': 'Tram', # Trams
    'T': 'Tram',
    'TRAM': 'Tram',
    'ATZ': 'ARZ', #AutoZug
    'D': 'RE', # Regional
    'RB': 'R',
    'M': 'Metro', # Metro
    'ICE': 'IC', # InterCityExpress, but routes.txt doesn't have that category
    'IRE': 'IR', # InterRegioExpress, but routes.txt doesn't have that category
    'BN': '', # Night
    'TN': '',
    'SN': '',
    'BT': '',
    'VAE': '', # Panorama trains in the Alps
    'PE': '',
    'TER': '', # France
    'TE2': '',
    'RJX': '', # International
    'null': '', # Other
    '': ''
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
@udf("string")
def replace_verkehrsmittel_text(text):
    if text in replace_actual.keys():
        return replace_actual[text]
    else:
        return text

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')\
                                 .where((col('bpuic').isin(stops_zurich)))\
                                 .where((col("ankunftszeit") != "") & (col("an_prognose") != ""))\
                                 .select(col('bpuic').alias('stop_id'),\
                                         replace_verkehrsmittel_text(col('verkehrsmittel_text')).alias('verkehrsmittel_text'),\
                                         from_unixtime(unix_timestamp('ankunftszeit', 'dd.MM.yyy HH:mm')).alias('ankunftszeit'),\
                                         from_unixtime(unix_timestamp('an_prognose', 'dd.MM.yyy HH:mm:ss')).alias('an_prognose'),
                                         col('an_prognose_status'))\
                                 .where(col("verkehrsmittel_text") != "")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+-------------------+------------------+
|stop_id|verkehrsmittel_text|       ankunftszeit|        an_prognose|an_prognose_status|
+-------+-------------------+-------------------+-------------------+------------------+
|8503000|                 EC|2018-09-03 21:51:00|2018-09-03 21:53:40|              REAL|
|8503000|                 EC|2018-09-03 10:51:00|2018-09-03 10:51:28|              REAL|
|8503000|                 IC|2018-09-03 07:00:00|2018-09-03 07:00:01|              REAL|
|8503000|                 IC|2018-09-03 21:23:00|2018-09-03 21:24:55|              REAL|
|8503000|                 IC|2018-09-03 08:26:00|2018-09-03 08:28:06|              REAL|
|8503000|                 IC|2018-09-03 17:53:00|2018-09-03 17:55:21|              REAL|
|8503000|                 IC|2018-09-03 10:00:00|2018-09-03 09:59:07|              REAL|
|8503000|                 EC|2018-09-03 12:51:00|2018-09-03 12:52:50|              REAL|
|8503000|            

In [10]:
actual = actual.withColumn('hour', hour(col('ankunftszeit')))\
               .withColumn('diff', unix_timestamp('an_prognose') - unix_timestamp('ankunftszeit'))\
               .select(col('stop_id'), col('verkehrsmittel_text'), col('an_prognose_status'), col('hour'), col('diff'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
actual = actual.where((col('hour') >= 8) & (col('hour') <= 20))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+------------------+----+----+
|stop_id|verkehrsmittel_text|an_prognose_status|hour|diff|
+-------+-------------------+------------------+----+----+
|8503000|                 EC|              REAL|  10|  28|
|8503000|                 IC|              REAL|   8| 126|
|8503000|                 IC|              REAL|  17| 141|
|8503000|                 IC|              REAL|  10| -53|
|8503000|                 EC|              REAL|  12| 110|
|8503000|                 IC|              REAL|   8| 332|
|8503000|                 IC|              REAL|  17|   6|
|8503000|                 IC|              REAL|  18| -14|
|8503000|                 IC|              REAL|   8| 112|
|8503000|                 IC|              REAL|   8|  90|
|8503000|                 IC|              REAL|   8| -59|
|8503000|                 IC|              REAL|   9|  48|
|8503000|                 IC|              REAL|   9| 916|
|8503000|                 IC|              REAL|  10| -2

In [13]:
enough_real_values = actual.where(col('an_prognose_status') == "REAL")\
                        .groupBy('stop_id', 'verkehrsmittel_text', 'hour')\
                        .agg(count('diff').alias('count'))\
                        .withColumn("enough_values", col("count") >= 10)\
                        .select(col('stop_id').alias('stop_id2'), col('verkehrsmittel_text').alias('verkehrsmittel_text2'), col('hour').alias('hour2'), col('enough_values'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
actual = actual.join(enough_real_values, (actual.stop_id == enough_real_values.stop_id2) &\
                                        (actual.verkehrsmittel_text == enough_real_values.verkehrsmittel_text2) &\
                                        (actual.hour == enough_real_values.hour2))\
                    .select('stop_id', 'verkehrsmittel_text', 'an_prognose_status', 'hour', 'diff', 'enough_values')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
actual = actual.where((~(col("enough_values"))) | (col("an_prognose_status") == "REAL"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Sanity checks

In [ ]:
actual.where(col("enough_values")).select("an_prognose_status").distinct().show()

In [ ]:
actual.where(~(col("enough_values"))).select("an_prognose_status").distinct().show()

In [16]:
actual = actual.groupBy('stop_id', 'verkehrsmittel_text', 'hour').agg(mean("diff").alias("mean"), stddev("diff").alias("std"), count("diff").alias('number_of_records'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
actual = actual.where(col('number_of_records') >= 5).select('stop_id', 'verkehrsmittel_text', 'hour', 'mean', 'std')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
actual = actual.na.fill(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
actual.cache()

In [ ]:
actual.show()

In [19]:
@udf("float")
def compute_delay_percentile(mean, std, percentile):
    X = norm(loc=mean, scale=std)
    return float(X.ppf(percentile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
actual = actual.withColumn('p_90', compute_delay_percentile(col('mean'), col('std'), lit(0.9)))\
                .withColumn('p_91', compute_delay_percentile(col('mean'), col('std'), lit(0.91)))\
                .withColumn('p_92', compute_delay_percentile(col('mean'), col('std'), lit(0.92)))\
                .withColumn('p_93', compute_delay_percentile(col('mean'), col('std'), lit(0.93)))\
                .withColumn('p_94', compute_delay_percentile(col('mean'), col('std'), lit(0.94)))\
                .withColumn('p_95', compute_delay_percentile(col('mean'), col('std'), lit(0.95)))\
                .withColumn('p_96', compute_delay_percentile(col('mean'), col('std'), lit(0.96)))\
                .withColumn('p_97', compute_delay_percentile(col('mean'), col('std'), lit(0.97)))\
                .withColumn('p_98', compute_delay_percentile(col('mean'), col('std'), lit(0.98)))\
                .withColumn('p_99', compute_delay_percentile(col('mean'), col('std'), lit(0.99)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Save to hdfs:

In [21]:
actual.write.format("orc").mode('overwrite').save("/user/{}/delay_distribution_percentiles.orc".format(username))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Temp:

In [10]:
actual.where(col('stop_id') == '858734901').select('stop_id', 'haltestellen_name').show(2, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------------------+-------------------+----------------------+
|stop_id  |verkehrsmittel_text|ankunftszeit       |an_prognose        |haltestellen_name     |
+---------+-------------------+-------------------+-------------------+----------------------+
|858734901|Trm                |2018-10-15 23:39:00|2018-10-15 23:40:14|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-15 05:54:00|2018-10-15 05:54:54|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-15 07:09:00|2018-10-15 07:09:54|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-15 23:54:00|2018-10-15 23:57:40|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-16 00:09:00|2018-10-16 00:13:34|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-15 06:09:00|2018-10-15 06:13:28|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-15 07:24:00|2018-10-15 07:26:39|Zürich, Bahnhofquai/HB|
|858734901|Trm                |2018-10-16 00:24:00

# Compute mean and std arrival delay:

In [3]:
actual = spark.read.format('orc').load('/data/sbb/orc/istdaten/')\
                                 .where((col("ankunftszeit") != "") & (col("an_prognose") != "") & (col("verkehrsmittel_text") != ""))\
                                 .select(col('bpuic').alias('stop_id'),\
                                         col('verkehrsmittel_text').alias('verkehrsmittel_text'),\
                                         from_unixtime(unix_timestamp('ankunftszeit', 'dd.MM.yyy HH:mm')).alias('ankunftszeit'),\
                                         from_unixtime(unix_timestamp('an_prognose', 'dd.MM.yyy HH:mm:ss')).alias('an_prognose'),\
                                         col('haltestellen_name'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
actual.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+-------------------+-----------------+
|stop_id|verkehrsmittel_text|       ankunftszeit|        an_prognose|haltestellen_name|
+-------+-------------------+-------------------+-------------------+-----------------+
|8503424|                 RB|2018-09-03 05:54:00|2018-09-03 05:55:10|     Schaffhausen|
|8503424|                 RB|2018-09-03 06:24:00|2018-09-03 06:24:23|     Schaffhausen|
|8503424|                 RB|2018-09-03 06:54:00|2018-09-03 06:55:09|     Schaffhausen|
|8503424|                 RB|2018-09-03 07:24:00|2018-09-03 07:24:48|     Schaffhausen|
|8503424|                 RB|2018-09-03 07:54:00|2018-09-03 07:55:26|     Schaffhausen|
|8503424|                 RB|2018-09-03 08:24:00|2018-09-03 08:24:31|     Schaffhausen|
|8503424|                 RB|2018-09-03 08:54:00|2018-09-03 08:56:22|     Schaffhausen|
|8503424|                 RB|2018-09-03 09:24:00|2018-09-03 09:24:36|     Schaffhausen|
|8503424|                 RB|201

In [13]:
actual.where(col('stop_id') == '8517377').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+-------------------+-------------------+------------------+
|stop_id|verkehrsmittel_text|       ankunftszeit|        an_prognose| haltestellen_name|
+-------+-------------------+-------------------+-------------------+------------------+
|8517377|                  S|2018-09-03 05:05:00|2018-09-03 05:05:34|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 05:35:00|2018-09-03 05:35:06|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 05:52:00|2018-09-03 05:52:43|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 06:05:00|2018-09-03 06:05:03|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 06:22:00|2018-09-03 06:23:15|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 06:35:00|2018-09-03 06:34:58|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 06:52:00|2018-09-03 06:53:16|Bibenlos-Sonnenhof|
|8517377|                  S|2018-09-03 07:05:00|2018-09-03 07:05:39|Bibenlos-Sonnenhof|
|8517377|            

In [5]:
@udf('long')
def leng(s):
    return len(s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
actual.withColumn('leng', leng(col('stop_id'))).select('leng').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|leng|
+----+
|   7|
|   6|
|   9|
+----+

In [7]:
actual.withColumn('leng', leng(col('stop_id'))).where(col('leng') == 6).select('stop_id', 'haltestellen_name').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|stop_id|   haltestellen_name|
+-------+--------------------+
| 85null|     Haut du village|
| 85null|                    |
| 85null|        Pattus (NOC)|
| 85null|Auvernier Littora...|
+-------+--------------------+

In [8]:
actual.withColumn('leng', leng(col('stop_id'))).where(col('leng') == 7).select('stop_id', 'haltestellen_name').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|stop_id|   haltestellen_name|
+-------+--------------------+
|8501120|            Lausanne|
|8500103|           Sorvilier|
|8588439|         Pully, port|
|8592150|   Lutry, Grand-Pont|
|8579249|Pully, Monts-de-P...|
|8591059|Zürich, Bahnhof E...|
|8506194|                Berg|
|8509350|       Celerina Staz|
|8575812|  Vacallo, S. Simone|
|8576278|Regensdorf, Furtb...|
|8594236|Rümlang, Hörnlist...|
|8590888|Wallisellen, Hall...|
|8595864|Cranves-Sales, Ve...|
|8579719|Ste-Croix, av. de...|
|8571989|Roggenburg, Missi...|
|8572083|Fontenais, sur Le...|
|8573725|     Islisberg, Dorf|
|8570561|    Epesses, village|
|8570212|Poliez-Pittet, vi...|
|8505634|Brione (Verzasca)...|
+-------+--------------------+
only showing top 20 rows

In [9]:
actual.withColumn('leng', leng(col('stop_id'))).where(col('leng') == 9).select('stop_id', 'haltestellen_name').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|  stop_id|   haltestellen_name|
+---------+--------------------+
|857961101|       Grand-Chézard|
|858734901|Zürich, Bahnhofqu...|
|857582901|    Pedrinate, Paese|
|859149802|   Pedrinate, Chiesa|
|859157501|Campione, Via Tag...|
|857624951|                Watt|
|858809650|Volketswil, Hofwisen|
|859063951|Greifensee, Tumig...|
|859057350|Dübendorf, Kunste...|
|859089051|  Wallisellen, Ifang|
|858736802|           Gérardmer|
|857959701|St-Sulpice NE Pri...|
|857181502|Le Cerneux-Péquig...|
|858370102|Le Cachot, Cachot...|
|857183202|Le Quartier, La C...|
|858239301|Delémont, zi Comm...|
|857677501|Les Genevez JU, c...|
|858367101|     Coeuve, Le Crêt|
|857204101|    Porrentruy, gare|
|850795802|   Romairon, village|
+---------+--------------------+
only showing top 20 rows